# GradientTape( )

In [1]:
import tensorflow as tf

In [5]:
x = tf.Variable(0.)
with tf.GradientTape() as tape:
    y = 2*x + 3

grad_of_y_wrt_x = tape.gradient(y, x)
grad_of_y_wrt_x

<tf.Tensor: shape=(), dtype=float32, numpy=2.0>

In [6]:
# 다차원의 텐서와 함께 사용하기
x = tf.Variable(tf.zeros((2, 2)))
with tf.GradientTape() as tape:
    y = 2*x + 3

grad_of_y_wrt_x = tape.gradient(y, x)
grad_of_y_wrt_x

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 2.],
       [2., 2.]], dtype=float32)>

In [7]:
# 변수 리스트의 그레이디언트 계산하기
w = tf.Variable(tf.random.uniform((2, 2)))
b = tf.Variable(tf.random.uniform((2,)))
x = tf.random.uniform((2,2))
with tf.GradientTape() as tape:
    y = tf.matmul(x, w) + b

grad_of_y_wrt_x = tape.gradient(y, [w, b])
grad_of_y_wrt_x

[<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
 array([[1.1593181, 1.1593181],
        [1.0146226, 1.0146226]], dtype=float32)>,
 <tf.Tensor: shape=(2,), dtype=float32, numpy=array([2., 2.], dtype=float32)>]

In [35]:
# Dense층 만들기
import tensorflow as tf

class NaiveDense:
    def __init__(self, input_size, output_size, activation):
        self.activation = activation
        
        w_shape = (input_size, output_size)
        w_initial_value = tf.random.uniform(w_shape, minval=0, maxval=0.1)
        self.w = tf.Variable(w_initial_value)
        
        b_shape = (output_size, )
        b_initial_value = tf.zeros(b_shape)
        self.b = tf.Variable(b_initial_value)
    
    def __call__(self, inputs):
        return self.activation(tf.matmul(inputs, self.w) + self.b)
    
    @property
    def weights(self):
        return [self.w, self.b]  # 리스트를 묶어서 리턴
    # @는 일종의 예약어 취급하기. 이건 특별한 약속을 하고 있다. 
    # 파이썬에서는 decoration이라고 하고 원래 함수는 weights( ) 이렇게 불렀는데
    # 이건 함수가 아니라 속성을 말하며 weights는 괄호를 안붙이고 부른다.

In [36]:
# Sequential 만들기
class NavieSequential:
    def __init__(self, layers):
        self.layers = layers  # 왼쪽은 새로 만들어지는 변수이다. layers를 받아서 새로운 변수에 넣어줌
    
    def __call__(self, inputs):
        x = inputs
        for layer in self.layers:
            x = layer(x)
        return x
    
    @property
    def weights(self):
        weights = []
        for layer in self.layers:
            weights += layer.weights  # layers들을 순환하면서 가중치를 업데이트 시켜준다.
        return weights

In [37]:
# 모델 구성하기
model = NavieSequential([
    NaiveDense(input_size=28*28, output_size=512, activation=tf.nn.relu),
    NaiveDense(input_size=512, output_size=10, activation=tf.nn.softmax)
])
# 512이 인 이유는 위에 512를 받아서 하는거여서.

In [38]:
assert len(model.weights) == 4
# assert는 중간중간에 확인을 하라는 건데 4면 계속 진행하고 아니면 멈춰라 라는것.

In [39]:
# 배치 제너레이터
import math

class BatchGenerator:
    def __init__(self, images, labels, batch_size=128):
        assert len(images) == len(labels)
        self.index = 0
        self.images = images
        self.labels = labels
        self.batch_size = batch_size
        self.num_batches = math.ceil(len(images) / batch_size)
        
    def next(self):
        images = self.images[self.index : self.index + self.batch_size]
        labels = self.labels[self.index : self.index + self.batch_size]
        self.index += self.batch_size
        return images, labels

In [40]:
# 훈련 스텝 실행하기
def one_training_step(model, images_batch, labels_batch):
    with tf.GradientTape() as tape:
        predictions = model(images_batch)
        per_sample_losses = tf.keras.losses.sparse_categorical_crossentropy(labels_batch, predictions)
        average_loss = tf.reduce_mean(per_sample_losses)
    gradients = tape.gradient(average_loss, model.weights)
    update_weights(gradients, model.weights)
    return average_loss

In [ ]:
learning_rate = 0.001
def update_weights(gradients, weights):
    for g, w in zip(gradients, weights):
        w.assign_sub(g * learning_rate)   # w = w - learning_rate*g 와 같다.
# 실제로는 이런 방법보단 optimizer 인스턴스를 주로 사용한다.

In [41]:
# Optimizer 인스턴스 사용하기
from tensorflow.keras import optimizers

optimizers = optimizers.SGD(learning_rate=1e-3)

def update_weights(gradients, weights):
    optimizers.apply_gradients(zip(gradients, weights))
# 이제 배치 훈련 스텝이 준비되었다.

In [42]:
# 전체 훈련 루프
def fit(model, images, labels, epochs, batch_size=128):
    for epoch_counter in range(epochs):
        print(f'에포크 {epoch_counter}')
        batch_generator = BatchGenerator(images, labels)
        for batch_counter in range(batch_generator.num_batches):
            images_batch, labels_batch = batch_generator.next()
            loss = one_training_step(model, images_batch, labels_batch)
            if batch_counter % 100 == 0:
                print(f'{batch_counter}번째 배치 손실: {loss:.2f}')

In [43]:
# 위의 함수를 테스트 하기
from tensorflow.keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28*28))
train_images = train_images.astype('float32') / 255
test_images = test_images.reshape((10000, 28*28))
test_images = test_images.astype('float32') / 255

fit(model, train_images, train_labels, epochs=10, batch_size=128)

에포크 0
0번째 배치 손실: 4.27
100번째 배치 손실: 2.23
200번째 배치 손실: 2.19
300번째 배치 손실: 2.06
400번째 배치 손실: 2.21
에포크 1
0번째 배치 손실: 1.91
100번째 배치 손실: 1.87
200번째 배치 손실: 1.81
300번째 배치 손실: 1.68
400번째 배치 손실: 1.81
에포크 2
0번째 배치 손실: 1.58
100번째 배치 손실: 1.57
200번째 배치 손실: 1.49
300번째 배치 손실: 1.39
400번째 배치 손실: 1.50
에포크 3
0번째 배치 손실: 1.32
100번째 배치 손실: 1.33
200번째 배치 손실: 1.23
300번째 배치 손실: 1.18
400번째 배치 손실: 1.27
에포크 4
0번째 배치 손실: 1.12
100번째 배치 손실: 1.15
200번째 배치 손실: 1.04
300번째 배치 손실: 1.02
400번째 배치 손실: 1.11
에포크 5
0번째 배치 손실: 0.98
100번째 배치 손실: 1.01
200번째 배치 손실: 0.90
300번째 배치 손실: 0.91
400번째 배치 손실: 0.99
에포크 6
0번째 배치 손실: 0.87
100번째 배치 손실: 0.91
200번째 배치 손실: 0.80
300번째 배치 손실: 0.82
400번째 배치 손실: 0.90
에포크 7
0번째 배치 손실: 0.79
100번째 배치 손실: 0.83
200번째 배치 손실: 0.72
300번째 배치 손실: 0.75
400번째 배치 손실: 0.84
에포크 8
0번째 배치 손실: 0.73
100번째 배치 손실: 0.76
200번째 배치 손실: 0.66
300번째 배치 손실: 0.70
400번째 배치 손실: 0.79
에포크 9
0번째 배치 손실: 0.68
100번째 배치 손실: 0.71
200번째 배치 손실: 0.61
300번째 배치 손실: 0.66
400번째 배치 손실: 0.74


In [44]:
# 모델 평가하기
import numpy as np

predictions = model(test_images)
predictions = predictions.numpy()
predicted_labels = np.argmax(predictions, axis=1)
matches = predicted_labels == test_labels
print(f'정확도: {matches.mean():.2f}')

정확도: 0.81


```
__call__( ) 의 역할
```

In [27]:
class Test:
    def __call__(self, num):  #call 매스를 숫자로 받는다.
        if num == 1:
            print('Hello')
        else:
            raise  # 아니라면 강제로 에러를 발생시키겠다.

In [30]:
test = Test()   # 이 함수를 test 라는 객체로 만들어주었다.

In [33]:
test(1)

Hello


In [34]:
test(9)

RuntimeError: No active exception to reraise

In [35]:
# 오브젝트를 만들지 않고 바로 넣어보기
Test()(1)

Hello
